In [1]:
##################################################################
# # ! Juno Magnetic Modelling using <Vector Potential> with <3 NNs>
# Table showing the RMS errors of `Spherical Harmonic Models` 
# computed at different subset of ***Juno Observation Orbits***
##################################################################
# %%
# # ! Setup
import numpy as np;
from LW_DataReadWrite import *;
from LW_CoordinateTransformation import *;
from LW_SH_Mag import *;

In [2]:
# %%

# # ! SH Model Estimations of [Bx,By,Bz] at <Multiple Different Obs Dataset> using different <nmax>
# TODO ******************** Parameters ******************** #
cstRj=71492;        # Jupiter radius in km
GS2nT=1e5;          # Gauss to nt
# TODO ********************** end ************************* #
######## Test Juno OBS dataset
FILEOBSs = ['input/Connerney_PJ01_33_4.0Rj.txt',      # <Connerney Obs dataset PJ01-33>
            'input/Juno_PJ01_33_4.0Rj.txt',           # <Our Obs dataset PJ01-33>
            'input/Juno_PJ01_50_4.0Rj.txt'];          # <Our Obs dataset PJ01-50>  
OBSNAMEs = ['CnyObs33',
          'OurObs33',
          'OurObs50'];
######## Interior SH models
FILESHs_INT = ['input/JRM33_I30.txt',
               'input/Bloxham_I32.txt'];
NMAXs = [30,32];    # Choose the expansion degree n
######## Exterior MagnetoDisc models
MDa = {'R0':7.8, 'R1':51.4, 'D':3.6, 'muI_2':139.6, 'md_lon':204.2, 'md_lat':9.3};
MDb = {'R0':5.0, 'R1':50.0, 'D':2.5, 'muI_2':225.0, 'md_lon':204.2, 'md_lat':9.3};
MAGDISCs  = [MDa,MDb];
######## Computation
SHNAMEs = ['JRM33_I30MDa','Bloxham_I32MDb'];
pkNMAXs = [[18,30],[18,32]];
for iModel in range(len(FILESHs_INT)):
     nmax =NMAXs[iModel];
     shName = SHNAMEs[iModel];
     fileSH_int = FILESHs_INT[iModel];
     MagDisc = MAGDISCs[iModel];
     print('\n*********************\n',flush=True);
     print('RMS of %s Model evaluated at Juno <OBS>:'%(shName),flush=True);
     for n in range(nmax):
          if ((n+1) in pkNMAXs[iModel]):
               print('\033[41mn = %2d; \033[0m'%(n+1),end='',flush=True);
          else:
               print('n = %2d; '%(n+1),end='',flush=True);
          for iFILEOBS in range(len(FILEOBSs)):
               fileObs = FILEOBSs[iFILEOBS];
               nObs,PJ,Year,DD,xObs,yObs,zObs,BxObs,ByObs,BzObs = LoadObsFile(fileObs,showinfo=False);
               xObs=xObs/cstRj; yObs=yObs/cstRj; zObs=zObs/cstRj;          # Distance in RJ
               BxObs=GS2nT*BxObs; ByObs=GS2nT*ByObs; BzObs=GS2nT*BzObs;    # Magnetic in nT
               refBNorm = np.sqrt(BxObs**2+ByObs**2+BzObs**2);
               if ((n+1) in pkNMAXs[iModel]):
                    print('\033[41m%9s, \033[0m'%(OBSNAMEs[iFILEOBS]),end='',flush=True);
               else:
                    print('%9s, '%(OBSNAMEs[iFILEOBS]),end='',flush=True);
               estBx_I,estBy_I,estBz_I,_ = SHS_Bxyz(fileSH_int,n+1,1.0,xObs,yObs,zObs,showinfo=False);
               estBx_I=GS2nT*estBx_I; estBy_I=GS2nT*estBy_I; estBz_I=GS2nT*estBz_I;    # Magnetic in nT
               mdXs, mdYs, mdZs = ecef2MD(xObs, yObs, zObs, MagDisc['md_lon'], MagDisc['md_lat']);
               mdBx, mdBy, mdBz, mdBNorm  = \
                    MagnetoDisc(mdXs, mdYs, mdZs, MagDisc['R0'], MagDisc['R1'], MagDisc['D'], MagDisc['muI_2']);
               estBx_E,estBy_E,estBz_E,_ = MD2ecef_v(mdBx, mdBy, mdBz, MagDisc['md_lon'], MagDisc['md_lat']);
               estBx = estBx_I + estBx_E;
               estBy = estBy_I + estBy_E;
               estBz = estBz_I + estBz_E;
               estBNorm = np.sqrt(estBx**2+estBy**2+estBz**2); 
               rms = np.sqrt(np.mean((estBNorm-refBNorm)**2));
               if ((n+1) in pkNMAXs[iModel]):
                    print('\033[41mrms = %8.1f nT; \033[0m'%(rms),end='',flush=True);
               else:
                    print('rms = %8.1f nT; '%(rms),end='',flush=True);
          print('',flush=True); # <br>


*********************

RMS of JRM33_I30MDa Model evaluated at Juno <OBS>:
n =  1;  CnyObs33, rms =  67236.3 nT;  OurObs33, rms =  66942.5 nT;  OurObs50, rms =  74718.9 nT; 
n =  2;  CnyObs33, rms =  49816.5 nT;  OurObs33, rms =  49626.7 nT;  OurObs50, rms =  56862.2 nT; 
n =  3;  CnyObs33, rms =  38646.8 nT;  OurObs33, rms =  38570.2 nT;  OurObs50, rms =  46366.7 nT; 
n =  4;  CnyObs33, rms =  30579.2 nT;  OurObs33, rms =  30528.2 nT;  OurObs50, rms =  36107.8 nT; 
n =  5;  CnyObs33, rms =  20428.5 nT;  OurObs33, rms =  20397.4 nT;  OurObs50, rms =  23839.8 nT; 
n =  6;  CnyObs33, rms =  14635.8 nT;  OurObs33, rms =  14567.4 nT;  OurObs50, rms =  17310.7 nT; 
n =  7;  CnyObs33, rms =  11894.7 nT;  OurObs33, rms =  11846.2 nT;  OurObs50, rms =  14133.1 nT; 
n =  8;  CnyObs33, rms =   8151.8 nT;  OurObs33, rms =   8122.7 nT;  OurObs50, rms =  10360.2 nT; 
n =  9;  CnyObs33, rms =   6232.7 nT;  OurObs33, rms =   6214.3 nT;  OurObs50, rms =   8058.4 nT; 
n = 10;  CnyObs33, rms =   4777.1 